In [1]:
import argparse
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
from torch.autograd import Variable
import os
import sys
sys.path.insert(0, './../')
from experiment_abstract.dataset import Dictionary, VQAFeatureDataset
import model_predict as base_model
print('ok')
os.chdir('./../')
exp = './experiment_abstract/'


torch.manual_seed(1111)
torch.cuda.manual_seed(1111)
torch.backends.cudnn.benchmark = True


dictionary = Dictionary.load_from_file(exp+'data/dictionary.pkl')
train_dset = VQAFeatureDataset('train', dictionary, dataroot=exp+'data/')
eval_dset = VQAFeatureDataset('val', dictionary, dataroot=exp+'data/')
batch_size = 512
num_hid = 1024

ok
loading dictionary from ./experiment_abstract/data/dictionary.pkl
loading features from h5 file
answer_path ./experiment_abstract/data/cache/train_target.pkl
loading features from h5 file
answer_path ./experiment_abstract/data/cache/val_target.pkl


In [2]:
constructor = 'build_baseline0_newatt'
model = getattr(base_model, constructor)(train_dset, num_hid).cuda()
model.w_emb.init_embedding(exp+'data/glove6b_init_300d.npy')
train_loader = DataLoader(train_dset, batch_size, shuffle=True, num_workers=1)
eval_loader =  DataLoader(eval_dset, batch_size, shuffle=True, num_workers=1)

In [3]:
model_path = exp+'saved_models/my_model_2/model.pth'
model_params = torch.load(model_path)

In [4]:
model_params.keys()[:3]

['module.w_emb.emb.weight',
 'module.q_emb.rnn.weight_ih_l0',
 'module.q_emb.rnn.weight_hh_l0']

In [5]:
model = nn.DataParallel(model).cuda()
model.load_state_dict(model_params)

In [6]:
model.eval()
pass

In [7]:
from train import train

In [8]:
train(model, train_loader, eval_loader, num_epochs=1, output=None)

train.py:65: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(model.parameters(), 0.25)
train.py:70: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  total_loss += loss.data[0] * v.size(0)


epoch 0, time: 39.24
	train_loss: 0.92, score: 91.38
	eval score: 70.47 (95.73)


In [11]:
model_params['module.classifier.main.0.bias']

tensor([ 0.0139,  0.0230,  0.0051,  ..., -0.0344, -0.0007,  0.0144], device='cuda:0')

In [31]:
model_params['module.classifier.main.3.weight_v'].grad#.sum(0)

In [12]:
model.module.classifier.main#[3].grad

Sequential(
  (0): Linear(in_features=2048, out_features=2048, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.5, inplace)
  (3): Linear(in_features=2048, out_features=426, bias=True)
)

In [35]:
params = list(model.parameters())

In [56]:
params[-4].grad.sum()#.shape

tensor(0., device='cuda:0')